# Single namespace in notebook?  Not anymore

Jupyter notebook is a great tool for prototying and exploring new ideas.  It's all good but falls short of one thing: namespace.  Although [an individual notebook may be imported as a module](http://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Importing%20Notebooks.html), within a notebook Jupyter doesn't tell you how to create a namespace.

So here is the way around: take advantage of a notebook cell and create modules using [IPython magic](http://ipython.readthedocs.io/en/stable/config/custommagics.html#defining-magics).  IPython magics provides a way to read the cell content.  The source code can be fed it into [exec](https://docs.python.org/3/library/functions.html#exec) and [imp](https://docs.python.org/2.7/library/imp.html#imp.new_module) and then produce a shining module.

In [1]:
# My module magics.

from __future__ import print_function


import os
import sys
import imp
import collections
import argparse

from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

from IPython.core.magic import Magics, magics_class, line_cell_magic
from IPython.display import display, HTML


# publish the CSS for pygments highlighting
display(HTML("""
<style type='text/css'>
%s
</style>
""" % HtmlFormatter().get_style_defs()
))


class ModuleContent(object):
    def __init__(self, fullname, mod, source, line=None):
        super(ModuleContent, self).__init__()
        self.fullname = fullname
        self.mod = mod
        self.source = source
        self.line = line
        
    @property
    def name(self):
        return fullname.split('.')[-1]

@magics_class
class ModuleMagics(Magics):
    """Turn a cell into a module."""
    
    def __init__(self, shell=None, **kw):
        if shell is None:
            shell = get_ipython()            
        super(ModuleMagics, self).__init__(shell, **kw)
        self.contents = collections.OrderedDict()

    def _create_parents(self, fullname):
        """Create parent module objects so that "from a.b import c" may be used."""
        names = fullname.split('.')
        for it in range(len(names)):
            pname = '.'.join(names[:it+1])
            if pname not in sys.modules:
                sys.modules[pname] = imp.new_module(pname)
        for it in range(len(names)-1, -1, -1):
            pname = '.'.join(names[:it])
            cname = '.'.join(names[:it+1])
            if pname:
                pmod = sys.modules[pname]
                setattr(pmod, names[it], sys.modules[cname])
        
    def _build_module(self, tokens, cell):
        """Build a module from cell.
        
        If multiple cells have tagged as the same module, those cells are incrementally 
        built into the module.  The source code is concatenated to the ModuleContent.
        
        Nested module is OK."""
        name = tokens[0]
        if len(tokens) == 1:
            fullname = tokens[0]
        elif len(tokens) == 3 and tokens[1] == 'in':
            fullname = '.'.join([tokens[2], tokens[0]])
        else:
            print('usage: %%module build name [in parent_package]')
            return

        # retrieve content module.
        content = self.contents.get(fullname, None)
        if not content:
            if len(tokens) == 1:
                line = '%%module build %s' % fullname
            else:
                line = '%%module build %s in %s' % (tokens[0], tokens[2])
            content = ModuleContent(fullname, imp.new_module(fullname), cell, line=line)
        else:
            content.source += cell
        mod = content.mod

        # compile the module.
        exec(cell, mod.__dict__)
        self.contents[fullname] = content
        
        # module namespace.
        sys.modules[fullname] = mod
        self._create_parents(fullname)

        # notebook shell namespace.
        shell_names = {name: mod}
        top_name = fullname.split('.')[0]
        if top_name != name:
            shell_names[top_name] = sys.modules[top_name]
        self.shell.push(shell_names)

    def _list_modules(self):
        """List all modules managed by this magic."""
        print(list(self.contents.keys()))

    def _show_module(self, name):
        """Show the module content."""
        ent = self.contents.get(name, None)
        if not ent:
            ent = dict((ent.name, ent.mod) for ent in self.contents).get(name, None)
        if ent:
            formatter = HtmlFormatter()
            lexer = PythonLexer()
            source = '# %s\n%s' % (ent.line, ent.source)
            html = highlight(source, lexer, formatter)
            display(HTML(html))
        else:
            print('no module named %s' % name)

    @line_cell_magic
    def module(self, line, cell=None):
        tokens = line.split()
        if cell is None:
            # %module list
            if tokens[0] == 'list':
                self._list_modules()
            # %module show mod_name
            elif len(tokens) == 2 and tokens[0] == 'show':
                self._show_module(tokens[1])
            else:
                print('usage: %module list')
                print('               show mod_name')
                return
        else:
            # %%module build ...
            if tokens[0] == 'build':
                self._build_module(tokens[1:], cell)
            else:
                print('usage: %%module build name [in parent_package]')
                return
            
get_ipython().register_magics(ModuleMagics)

# Build a module

Show-case building a simple module.

In [2]:
%%module build mod_a

top_attr = 'I am in mod_a'

def func():
    print("I am a function in %s" % __name__)

In [3]:
print(mod_a.top_attr)
mod_a.func()
print('mod_a source:')
%module show mod_a

I am in mod_a
I am a function in mod_a
mod_a source:


Contents in another cell can be added to an existing module.

In [4]:
%%module build mod_a

def second_function():
    print("I am the second function in %s" % __name__)
    
top_attr = 'module attribute is overridden'

In [5]:
mod_a.second_function()
print(mod_a.top_attr)
print('mod_a source, modified:')
%module show mod_a

I am the second function in mod_a
module attribute is overridden
mod_a source, modified:


# Nested module

In [6]:
%%module build submod in parent.supermod

def func_in_submod():
    print('I am func_in_submod in %s' % __name__)

In [7]:
assert 'submod' in globals()
submod.func_in_submod()
print(submod)
%module show parent.supermod.submod

I am func_in_submod in parent.supermod.submod
<module 'parent.supermod.submod'>


The top-level module is automatically brought into the notebook namespace.

In [8]:
assert 'parent' in globals()
parent.supermod.submod.func_in_submod()

I am func_in_submod in parent.supermod.submod


When using the dot-separated absolute module name, the module isn't automatically added to the notebook namespace.

In [9]:
%%module build parent.supermod.second_mod

def func_in_secondmod():
    print('I am the second_mod in %s' % __name__)

In [10]:
assert 'second_mod' not in globals()
parent.supermod.second_mod.func_in_secondmod()
%module show parent.supermod.second_mod

I am the second_mod in parent.supermod.second_mod


# References

## IPython magics

- Someone else's module magic.  I don't want to save a cell to a file and import again.
  https://github.com/brazilbean/modulemagic/blob/master/modulemagic/modulemagic.py
- An example of using magic to treat cell contents:
  https://stackoverflow.com/questions/33508377/read-cell-content-in-an-ipython-notebook

## Dynamic import

- http://www.pythondoeswhat.com/2011/12/easily-import-dynamically-created.html
- https://stackoverflow.com/questions/2931950/dynamic-module-creation